In [1]:
"""Import from parent directory."""
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path: sys.path.append(module_path)

In [1]:
import os

import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers

#from generic_model import GenericKerasModel
from models.generic_model import GenericKerasModel

class KerasPerceptron(GenericKerasModel):
    def __init__(self, is_training=False):
        self.n_input = 784
        self.n_hidden1 = 200
        self.n_hidden2 = 200
        self.n_classes = 10
        self.is_training = is_training
        self.model = self.build_model()
        if is_training:
            self.compile_model()

    def build_model(self):
        model = Sequential()
        model.add(Dense(self.n_hidden1, input_shape=(self.n_input,), activation='relu'))
        model.add(Dense(self.n_hidden2, activation='relu'))
        model.add(Dense(self.n_classes, activation='linear'))
        model.summary()
        return model

    def compile_model(self):
        sgd = optimizers.SGD(lr=0.01, decay=1e-6)
        self.model.compile(
            optimizer=sgd,
            loss='categorical_crossentropy',
            metrics=['acc']
        )

def _create_dataset_iterator(dataset_path, max_count, iter_type, batch_size, labeler):
    """
    Returns an iterator of batches of size B containing all features of the data.

    Assumes `dataset_path` is a path to a folder with multiple CSV files.
    """
    assert iter_type in ['train', 'test'], "'iter_type' parameter is invalid."
    if iter_type == 'train':
        directories = os.listdir(dataset_path)
    elif iter_type == 'test':
        directories = reversed(os.listdir(dataset_path))

    count = 0
    for filename in directories:
        if not filename.endswith(".csv"): continue
        full_path = os.path.join(dataset_path, filename)
        with open(full_path, 'r') as f:
            batch = []
            for line in f:
                if count >= max_count:
                    print('exit at {} and {}'.format(count, max_count))
                    return
                if len(batch) == batch_size:
                    if batch_size == 1:
                        yield batch[0]
                    else:
                        yield batch
                    batch = []
                line = labeler(line) if labeler else line
                batch.append(line)
                count += 1

def create_train_dataset_iterator(dataset_path, count, split=0.8, batch_size=1, labeler=None):
    """
    Returns an iterator of batches of size B containing all features of the data.

    Assumes `dataset_path` is a path to a folder with multiple CSV files.
    """
    while True:
        yield from _create_dataset_iterator(dataset_path, int(count*split), "train", \
            batch_size, labeler)
        
def create_test_dataset_iterator(dataset_path, count, split=0.2, batch_size=1, labeler=None):
    """
    Returns an iterator of batches of size B containing all features of the data.

    Assumes `dataset_path` is a path to a folder with multiple CSV files.
    """
    while True:
        yield from _create_dataset_iterator(dataset_path, count*split, "test", \
            batch_size, labeler)

def train_keras_model(serialized_model, weights, dataset_iterator, hyperparams):
    logging.info('Keras training just started.')
    assert weights != None, "Initial weights must not be 'None'."
    model = model_from_json(serialized_model)
    model.set_weights(weights)
    hist = model.fit_generator(dataset_iterator, epochs=hyperparams['epochs'])
    new_weights = model.get_weights()
    logging.info('Keras training complete.')
    return new_weights, {'training_history' : hist.history}

def mnist_labeler(line):
    """
    Returns an (X, y) tuple from a line of the MNIST CSV files, where X is a
    (784, 1) numpy array and y is a scalar.

    The input is of the format `label, pix-11, pix-12, pix-13, ...`.
    """
    line = line.split(',')
    X = np.array(line[1:], dtype=np.uint8).reshape((784,))
    y = int(line[0])
    assert X.size == 784, "Dimension of `X` is not 784."
    assert 0 <= y <= 9, "`y` is not between 0 and 9."
    X = np.expand_dims(X, axis=0)
    y = np.expand_dims(keras.utils.to_categorical(y, num_classes=10), axis=0)
    return X, y


/Users/georgymh/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data_count = 60000
split = 0.8
batch_size = 1
epochs = 2

m = KerasPerceptron(is_training=True)
dataset_iterator = create_train_dataset_iterator('datasets/mnist', data_count, \
    split=split, batch_size=batch_size, labeler=mnist_labeler)

# i = 1
# for X, y in dataset_iterator:
#     #print(len(b), len(b[0]), b[1])
#     print(X)
#     print(y)
#     i -= 1
#     if i == 0: exit(1)

# i = 0
# for b in _create_dataset_iterator('datasets/mnist', data_count*split, \
#     batch_size=batch_size, labeler=mnist_labeler):
#     i += 1
# i

# i = 0
# for b in dataset_iterator:
#     i += 1
# i

hist = m.model.fit_generator(dataset_iterator, epochs=epochs, \
    steps_per_epoch=(data_count*split)//batch_size)
hist.history

# # m.build_optimizer(1.47)
# # m.train(text, {"epochs":1, "batch_size":8})

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 200)               157000    
_________________________________________________________________
dense_2 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2010      
Total params: 199,210
Trainable params: 199,210
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
48000/48000 [==============================] - 133s 3ms/step - loss: 9.5261 - acc: 0.1046
Epoch 2/2
48000/48000 [==============================] - 150s 3ms/step - loss: 9.4576 - acc: 0.1047


{'acc': [0.1045625, 0.10466666666666667],
 'loss': [9.526110862900659, 9.457628317661584]}

In [3]:
hist.history

{'acc': [0.1045625, 0.10466666666666667],
 'loss': [9.526110862900659, 9.457628317661584]}

In [4]:
dataset_iterator = create_test_dataset_iterator('datasets/mnist', data_count, \
    split=split, batch_size=batch_size, labeler=mnist_labeler)

hist = m.model.evaluate_generator(dataset_iterator, steps=(data_count*(1-split))//batch_size)

print(hist)

[9.495692720710462, 0.1051754312859405]


In [5]:
d = dict(zip(m.model.metrics_names, hist))
d

{'acc': 0.1051754312859405, 'loss': 9.495692720710462}